In [2]:
%reload_ext autoreload
%autoreload 2

# Sentiment IMDB anlysis with Transformers form scratch
Each sample is a sequence of embedded words. Output two positive/negative class
Input:
* batch of different sample sentences
* sample is a sentence of dim t x embedding_size (where t is max sequence)

In [10]:
import warnings
warnings.filterwarnings('ignore')
from _context import src
from src.models.model_utils import device_selection
#from src.models.predict_model import ClassSequenceTransformer
#from src.models import TransformerClassification
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchtext import data, datasets, vocab

import numpy as np
import random, tqdm, sys, math, gzip


In [37]:
#Params
vocab_size = 50000
batch_size = 8
num_classes = 2
embeding_size = 128
transformer_heads = 8
depth = 4

## Dataset

In [12]:
text = data.Field(lower=True, include_lengths=True, batch_first=True)
label = data.Field(sequential=False)

In [13]:
train, test = datasets.IMDB.splits(text, label)
text.build_vocab(train, max_size=vocab_size - 2)
label.build_vocab(train)

In [14]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=batch_size, device=device_selection())

In [15]:
#Determine maximum sequence
max_sequence = max([seq.text[0].size(1) for seq in train_iter])

In [35]:
#Explore structure
for seq in train_iter:
    s = seq
    break
seq


[torchtext.data.batch.Batch of size 8 from IMDB]
	[.text]:('[torch.LongTensor of size 8x633]', '[torch.LongTensor of size 8]')
	[.label]:[torch.LongTensor of size 8]

# Classify Transformer

* Seq-to-seq model that does an global average pooling, the resulting vector is projected to a smaller dimension and  at the end and apply a softmax

* Positional embeding: each transformer is positional invariant (different order of the inputs produce the same output). Add positional information to the input vectrors by generating positional vectors that are added to the input words. Positional embeding let the model to learn the vector value for position ( does not work well in  sequence lenghts the network has nott seen before). Positional encoding just add a funtion that map position to a vectors, but the network does not learn the mapping, it is an complex hyperparam to fix.

<img src="images/transformer_model.svg"  width="500" height="600">



In [36]:
model = ClassSequenceTransformer(
                                    num_classes=num_classes,
                                    embeding_size=embeding_size,
                                    transformer_heads=transformer_heads,
                                    depth=depth,
                                    vocab_size=vocab_size,
                                    max_sequence=max_sequence,
                                    
                                )